In [1]:
from langchain_core.runnables import RunnablePassthrough,RunnableLambda, Runnable, RunnableParallel,RunnableConfig,RunnableGenerator
from langchain_core.messages import AIMessage
from dotenv import load_dotenv,find_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from langchain.tools import tool
from langchain.prompts import ChatPromptTemplate,SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from operator import itemgetter
from langchain.embeddings import SentenceTransformerEmbeddings
import json
from langchain_core.output_parsers import StrOutputParser
from langchain_community.vectorstores import FAISS,Chroma
from operator import itemgetter
import time
import grandalf
from typing import Iterator,List,AsyncIterator
from langchain_core.runnables import ConfigurableField
from langchain.runnables.hub import HubRunnable

C:\Users\MSI 1\.conda\envs\oaivenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv(find_dotenv("D:\LLM Courses\Master Langchain Udemy\.env"))

True

In [3]:
llm=ChatGoogleGenerativeAI(model="gemini-1.5-flash")

<h3> Configurable Fields </h3>

In [4]:
model=ChatOpenAI(temperature=0)\
    .configurable_fields\
        (
            temperature=ConfigurableField
                            (
                                id="llmTemperature",
                                name="LLM Temperature",
                                description="The temperature of the LLM"
                            )
        )

In [5]:
model.invoke(input="Select a Random Number between 1 and 10")

AIMessage(content='7', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 17, 'total_tokens': 18, 'completion_tokens_details': {'reasoning_tokens': 0}, 'prompt_tokens_details': {'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-76b9e37d-9f22-473d-9734-49a23b30c87a-0', usage_metadata={'input_tokens': 17, 'output_tokens': 1, 'total_tokens': 18})

In [6]:
StrOutputParser().invoke(model.invoke(input="Select a Random Number"))

'27'

In [7]:
model.with_config(config={"llmTemperature":0.3}).invoke(input="Select a Random Number")

AIMessage(content='27', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 11, 'total_tokens': 12, 'completion_tokens_details': {'reasoning_tokens': 0}, 'prompt_tokens_details': {'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-7fc20648-9a9a-4541-8088-6e1b98c54f46-0', usage_metadata={'input_tokens': 11, 'output_tokens': 1, 'total_tokens': 12})

In [8]:
prompt=ChatPromptTemplate.from_template(template="Pick a random Number above {x}")

In [9]:
chain=prompt|llm|StrOutputParser()

In [10]:
chain.with_config(config={"llmTemperature":0.2}).invoke(input={"x":23})

"I can't pick a truly random number as I'm a language model. However, I can generate a random number for you using a random number generator. \n\nThe random number above 23 is: **37** \n"

<h3>Hub Runnable</h3>

In [11]:
prompt=HubRunnable(owner_repo_commit="rlm/rag-prompt").configurable_fields(
    owner_repo_commit=ConfigurableField(
        id="hub_commit",
        name="Hub Commit",
        description="The Hub Commit to Pull from"
    )
)

C:\Users\MSI 1\.conda\envs\oaivenv\Lib\site-packages\langsmith\client.py:323: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [12]:
print(prompt.default.bound.messages[0].prompt.template)

You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:


In [86]:
print(prompt.invoke(input={"question":"foo","context":"bar"}).messages[0].content)

You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: foo 
Context: bar 
Answer:


In [13]:
print(prompt.with_config(
    config={"hub_commit":"rlm/rag-prompt-llama"}
).invoke(input={"question":"foo","context":"bar"}).messages[0].content)

C:\Users\MSI 1\.conda\envs\oaivenv\Lib\site-packages\langsmith\client.py:323: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


[INST]<<SYS>> You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.<</SYS>> 
Question: foo 
Context: bar 
Answer: [/INST]


<h3> Configurable Alternatives</h3>

In [14]:
llm=ChatGoogleGenerativeAI(temperature=0,model="gemini-1.5-flash").configurable_alternatives(
    which=ConfigurableField(id="llm"),
    default_key="gemini",
    openai=ChatOpenAI(model="gpt-3.5-turbo",temperature=0)
    ).configurable_fields(temperature=ConfigurableField(
            id="llmTemperature",
            name="LLM Temperature",
            description="The temperature of the LLM"  
))

In [15]:
prompt=ChatPromptTemplate.from_template(
    template="Tell me a joke about {topic}"
)

chain=prompt|llm

In [16]:
chain.invoke(input={"topic":"Giraffe"})

AIMessage(content='Why did the giraffe get a job at the library? \n\nBecause he was really good at reaching for the high shelves! 🦒📚 \n', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-d99741d2-fbd0-48bd-9b05-4011660aa610-0', usage_metadata={'input_tokens': 8, 'output_tokens': 28, 'total_tokens': 36})

In [17]:
chain.with_config(
    configurable={"llm":"openai","llmTemperature":0.6}
).invoke(input={"topic":"Giraffe"})

AIMessage(content='Why did the giraffe break up with the zebra? Because he was always telling tall tales!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 14, 'total_tokens': 34, 'completion_tokens_details': {'reasoning_tokens': 0}, 'prompt_tokens_details': {'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-aafcc223-3ee1-4608-9f9e-86050c13eec5-0', usage_metadata={'input_tokens': 14, 'output_tokens': 20, 'total_tokens': 34})

In [18]:
chain.with_config(
    configurable={"llm":"gemini","llmTemperature":0.3}
).invoke(input={"topic":"Giraffe"})

AIMessage(content='Why did the giraffe get a job at the library? \n\nBecause he was really good at reaching for the high shelves! \n', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-bbe83398-bc79-408d-be86-88a2ac7ba9c3-0', usage_metadata={'input_tokens': 8, 'output_tokens': 25, 'total_tokens': 33})

<h3>With Prompts</h3>

In [19]:
prompt=ChatPromptTemplate.from_template(
    template="Tell me a joke about {topic}"
).configurable_alternatives(
    ConfigurableField(id="prompt"),
    default_key="joke",
    poem=ChatPromptTemplate.from_template(
            template="Tell me a poem about {topic}"
    )
)

In [20]:
chain=prompt|llm

In [21]:
chain.with_config(
    configurable={"llm":"openai","llmTemperature":0.6,'prompt':'poem'}
).invoke(input={"topic":"Persistence"})

AIMessage(content='In the face of trials and tribulations,\nPersistence is our guiding light,\nThrough the darkest of situations,\nIt keeps us pushing with all our might.\n\nWhen obstacles block our way,\nAnd failure seems imminent,\nPersistence helps us stay,\nAnd continue our ascent.\n\nIt\'s the fuel that drives us forward,\nWhen all hope seems lost,\nIt\'s the voice that whispers, "You can do it",\nNo matter the cost.\n\nWith persistence as our ally,\nWe can conquer any foe,\nNo challenge is too daunting,\nNo mountain too high to go.\n\nSo let us hold onto this virtue,\nAnd never let it fade,\nFor with persistence as our compass,\nSuccess will never evade.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 135, 'prompt_tokens': 13, 'total_tokens': 148, 'completion_tokens_details': {'reasoning_tokens': 0}, 'prompt_tokens_details': {'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop'

In [23]:
openAIwithPoemPrompt=chain.with_config(
    configurable={"llm":"gemini","llmTemperature":0.6,'prompt':'poem'}
)

In [24]:
print(openAIwithPoemPrompt.invoke(input={"topic":"Zen"}).content)

The wind whispers through bamboo, a gentle, rustling sound,
A silent symphony of nature, on hallowed, sacred ground.
A single leaf falls, adrift on the breeze, a moment's fleeting grace,
A reminder of impermanence, in time and in this space.

A weathered stone sits, moss-covered, a witness to the years,
Unmoved by storms that rage around, unburdened by fears.
A simple cup of tea, held gently, the steam curls to the sky,
Each sip a meditation, on the present, here and now, why?

The mind, a restless monkey, leaps from thought to thought,
But Zen invites us inward, to stillness, to be caught
In the quiet of the moment, where truth is found at last,
In the breath, the heartbeat, the present, forever and amassed.

No need for grand pronouncements, no need for words to speak,
Just the quiet understanding, a knowing, deep and meek.
The path is paved with silence, with mindful steps we tread,
In the stillness of our being, the true self is instead.

So breathe in the cool air, let go of all y